<a href="https://colab.research.google.com/github/gilhotraapoorva/DF/blob/main/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"apoorvaaaaaa","key":"d66784d4565981100258ecf8a750e6a2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list


ref                                                  title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                  Online Food Dataset                                  3KB  2024-03-02 18:50:30           9993        185  0.9411765        
syedanwarafridi/vehicle-sales-data                   Vehicle Sales Data                                  19MB  2024-02-21 20:16:17          19493        339  1.0              
julianoorlandi/spotify-top-songs-and-audio-features  Spotify Top Songs and Audio Features               526KB  2024-03-18 12:49:50           1379         27  1.0              
bhavikjikadara/student-study-performance             Student Study Performance                            9KB  2024-03-0

In [ ]:
!kaggle competitions download -c deepfake-detection-challenge

deepfake-detection-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! mkdir train

In [ ]:
! unzip deepfake-detection-challenge.zip -d train

Archive:  deepfake-detection-challenge.zip
  inflating: train/sample_submission.csv  
  inflating: train/test_videos/aassnaulhq.mp4  
  inflating: train/test_videos/aayfryxljh.mp4  
  inflating: train/test_videos/acazlolrpz.mp4  
  inflating: train/test_videos/adohdulfwb.mp4  
  inflating: train/test_videos/ahjnxtiamx.mp4  
  inflating: train/test_videos/ajiyrjfyzp.mp4  
  inflating: train/test_videos/aktnlyqpah.mp4  
  inflating: train/test_videos/alrtntfxtd.mp4  
  inflating: train/test_videos/aomqqjipcp.mp4  
  inflating: train/test_videos/apedduehoy.mp4  
  inflating: train/test_videos/apvzjkvnwn.mp4  
  inflating: train/test_videos/aqrsylrzgi.mp4  
  inflating: train/test_videos/axfhbpkdlc.mp4  
  inflating: train/test_videos/ayipraspbn.mp4  
  inflating: train/test_videos/bcbqxhziqz.mp4  
  inflating: train/test_videos/bcvheslzrq.mp4  
  inflating: train/test_videos/bdshuoldwx.mp4  
  inflating: train/test_videos/bfdopzvxbi.mp4  
  inflating: train/test_videos/bfjsthfhbd.mp4  
  

In [1]:
import os

In [2]:
TRAIN_DIR = '/content/train/train_sample_videos/'
TMP_DIR = '/content/train/tmp/'
ZIP_NAME = 'dfdc_train_faces_sample.zip'
METADATA_PATH = TRAIN_DIR + 'metadata.json'

SCALE = 0.25
N_FRAMES = None

In [3]:
!pip install facenet-pytorch > /dev/null 2>&1
!apt install zip > /dev/null 2>&1

In [4]:
import os
import glob
import json
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

Running on device: cpu


In [5]:
class FaceExtractor:
    def __init__(self, detector, n_frames=None, resize=None):
        """
        Parameters:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """
        self.detector = detector
        self.n_frames = n_frames
        self.resize = resize

    def __call__(self, filename, save_dir):
        """Load frames from an MP4 video, detect faces and save the results.

        Parameters:
            filename {str} -- Path to video.
            save_dir {str} -- The directory where results are saved.
        """

        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)

                # Resize frame to desired size
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])

                save_path = os.path.join(save_dir, f'{j}.png')

                self.detector([frame], save_path=save_path)

        v_cap.release()

In [6]:
with open(METADATA_PATH, 'r') as f:
    metadata = json.load(f)


In [7]:
train_df = pd.DataFrame(
    [
        (video_file, metadata[video_file]['label'], metadata[video_file]['split'], metadata[video_file]['original'] if 'original' in metadata[video_file].keys() else '')
        for video_file in metadata.keys()
    ],
    columns=['filename', 'label', 'split', 'original']
)

train_df.head()

,filename,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [8]:
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()

In [9]:
face_extractor = FaceExtractor(detector=face_detector, n_frames=N_FRAMES, resize=SCALE)

In [10]:

all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))

In [11]:
!mkdir -p $TMP_DIR


In [12]:
import random
num_videos_to_process = 2
selected_videos = random.sample(all_train_videos, num_videos_to_process)

with torch.no_grad():
    for path in tqdm(selected_videos, desc="Processing videos"):
        file_name = os.path.basename(path)
        save_dir = os.path.join(TMP_DIR, os.path.splitext(file_name)[0])

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Detect all faces appear in the video and save them.
        face_extractor(path, save_dir)

Processing videos:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
cd $TMP_DIR


/content/train/tmp


In [14]:
train_df.to_csv('metadata.csv', index=False)


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [16]:
image_dimensions = {'height':256,'width':256,'channels':3}

In [17]:
class Classifier:
  def __init__():
    self.model = 0

  def predict(self,x):
    return self.model.predict(x)

  def fit(self,x,y):
    return self.model.train_on_batch(x,y)

  def get_accuracy(self,x,y):
    return self.model.test_on_batch(x,y)

  def load(self,path):
    self.model.load_weights(path)

In [18]:
class Meso4(Classifier):
  def __init__(self,learning_rate = 0.001):
    self.model = self.init_model()
    optimizer = Adam(lr = learning_rate)
    self.model.compile(optimizer = optimizer, loss = 'mean_squared_error',metrics = ['accuracy'])
  def init_model(self):
      x = Input(shape = (image_dimensions['height'],
                          image_dimensions['width'],
                          image_dimensions['channels']))

      x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
      x1 = BatchNormalization()(x1)
      x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

      x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
      x2 = BatchNormalization()(x2)
      x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

      x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
      x3 = BatchNormalization()(x3)
      x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

      x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
      x4 = BatchNormalization()(x4)
      x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

      y = Flatten()(x4)
      y = Dropout(0.5)(y)
      y = Dense(16)(y)
      y = LeakyReLU(alpha=0.1)(y)
      y = Dropout(0.5)(y)
      y = Dense(1, activation = 'sigmoid')(y)

      return Model(inputs = x, outputs = y)

In [19]:
dataGenerator = ImageDataGenerator(rescale=1./255)
generator = dataGenerator.flow_from_directory(
    '/content/train/tmp',
    target_size=(256, 256),
    batch_size=1,
    class_mode='binary')

Found 18511 images belonging to 58 classes.


In [ ]:
!rmdir /s /q c:data\.ipynb_checkpoints

In [21]:
meso = Meso4()

In [ ]:
# Rendering image X with label y for MesoNet
X, y = generator.next()
print(X)
# Evaluating prediction
print(f"Predicted likelihood: {meso.predict(X)[0][0]:.4f}")
print(f"Actual label: {int(y[0])}")
print(f"\nCorrect prediction: {round(meso.predict(X)[0][0])==y[0]}")

# Showing image
plt.imshow(np.squeeze(X));

In [23]:
correct_real = []
correct_real_pred = []

correct_deepfake = []
correct_deepfake_pred = []

misclassified_real = []
misclassified_real_pred = []

misclassified_deepfake = []
misclassified_deepfake_pred = []

In [ ]:
for i in range(len(generator.labels)):

    # Loading next picture, generating prediction
    X, y = generator.next()
    pred = meso.predict(X)[0][0]

    # Sorting into proper category
    if round(pred)==y[0] and y[0]==1:
        correct_real.append(X)
        correct_real_pred.append(pred)
    elif round(pred)==y[0] and y[0]==0:
        correct_deepfake.append(X)
        correct_deepfake_pred.append(pred)
    elif y[0]==1:
        misclassified_real.append(X)
        misclassified_real_pred.append(pred)
    else:
        misclassified_deepfake.append(X)
        misclassified_deepfake_pred.append(pred)

    # Printing status update
    if i % 1000 == 0:
        print(i, ' predictions completed.')

    if i == len(generator.labels)-1:
        print("All", len(generator.labels), "predictions completed")

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 42ms/step
9000  predictions completed.
1/1 [==============================] - 0s 41ms/step
10000  predictions completed.
1/1 [==============================] - 0s 41ms/step
11000  predictions completed.
1/1 [==============================] - 0s 41ms/step
12000  predictions completed.
1/1 [==============================] - 0s 64ms/step
13000  predictions completed.
1/1 [==============================] - 0s 51ms/step


In [ ]:
def plotter(images,preds):
    fig = plt.figure(figsize=(16,9))
    subset = np.random.randint(0, len(images)-1, 12)
    for i,j in enumerate(subset):
        fig.add_subplot(3,4,i+1)
        plt.imshow(np.squeeze(images[j]))
        plt.xlabel(f"Model confidence: \n{preds[j]:.4f}")
        plt.tight_layout()
        ax = plt.gca()
        ax.axes.xaxis.set_ticks([])
        ax.axes.yaxis.set_ticks([])
    plt.show;
    return

In [ ]:
plotter(correct_real, correct_real_pred)


In [ ]:
plotter(misclassified_real, misclassified_real_pred)


In [ ]:
plotter(correct_deepfake, correct_deepfake_pred)


In [ ]:
plotter(misclassified_deepfake, misclassified_deepfake_pred)


**END**

In [ ]:
len(os.listdir('/content/train/test_videos'))

400

In [ ]:
len(os.listdir('/content/train/train_sample_videos'))

401

In [ ]:
!pip install opencv-python

In [ ]:
!git clone https://github.com/ultralytics/ultralytics/yolov8

Cloning into 'yolov8'...
remote: Not Found
fatal: repository 'https://github.com/ultralytics/ultralytics/yolov8/' not found


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 985.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)


100%|██████████| 6.23M/6.23M [00:00<00:00, 72.1MB/s]


In [ ]:
model.train(data="/content/train/train_sample_videos", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set